# Autogluon fraud

김보람  
2023-10-12

In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [14]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor

# directory = '~/IEEEfraud/'  # directory where you have downloaded the data CSV files from the competition
label = 'isFraud'  # name of target variable to predict in this competition
eval_metric = 'roc_auc'  # Optional: specify that competition evaluation metric is AUC
# save_path = directory + 'AutoGluonModels/'  # where to store trained models

In [2]:
!kaggle competitions download -c ieee-fraud-detection

 99%|███████████████████████████████████████▌| 117M/118M [00:09<00:00, 18.2MB/s]
100%|████████████████████████████████████████| 118M/118M [00:09<00:00, 13.5MB/s]

In [4]:
!unzip ieee-fraud-detection.zip -d ./ieee-fraud-detection

Archive:  ieee-fraud-detection.zip
  inflating: ./ieee-fraud-detection/sample_submission.csv  
  inflating: ./ieee-fraud-detection/test_identity.csv  
  inflating: ./ieee-fraud-detection/test_transaction.csv  
  inflating: ./ieee-fraud-detection/train_identity.csv  
  inflating: ./ieee-fraud-detection/train_transaction.csv  

In [15]:
train_identity = pd.read_csv('ieee-fraud-detection/train_identity.csv')
train_transaction = pd.read_csv('ieee-fraud-detection/train_transaction.csv')

In [8]:
# !kaggle competitions download -c ieee-fraud-detection
# !unzip titanic.zip -d ./ieee-fraud-detection
# df_train = pd.read_csv('titanic/train.csv')
# df_test = pd.read_csv('titanic/test.csv')
# !rm titanic.zip
# !rm -rf titanic/

In [16]:
train_data = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')

In [17]:
train_data

In [18]:
predictor = TabularPredictor(label=label, eval_metric=eval_metric, verbosity=3).fit(
    train_data, presets='best_quality', time_limit=1200
)

results = predictor.fit_summary()

No path specified. Models will be saved in: "AutogluonModels/ag-20231012_071002"
Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': True}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': True,
 'calibrate': 'auto',
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'included_model_types': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': None,
 'num_bag_sets': None,
 'num_stack_levels': None,
 'pseudo_data': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 3}
/home/coco/anaconda3/envs/ag/lib/python3.10/site-packages/autogluon/core/utils/utils.py:564: FutureWarning: use_inf_as_na option is depreca

[50]    valid_set's binary_logloss: 0.0936737
[100]   valid_set's binary_logloss: 0.0852533
[150]   valid_set's binary_logloss: 0.0807703
[200]   valid_set's binary_logloss: 0.0774437
[250]   valid_set's binary_logloss: 0.0748597
[300]   valid_set's binary_logloss: 0.072922
[350]   valid_set's binary_logloss: 0.0713176
[400]   valid_set's binary_logloss: 0.0698932
[450]   valid_set's binary_logloss: 0.068628
[500]   valid_set's binary_logloss: 0.0675102
[550]   valid_set's binary_logloss: 0.0664848
[600]   valid_set's binary_logloss: 0.0656376
[650]   valid_set's binary_logloss: 0.064794
[700]   valid_set's binary_logloss: 0.063985
[750]   valid_set's binary_logloss: 0.0632608
[800]   valid_set's binary_logloss: 0.0625744
[850]   valid_set's binary_logloss: 0.0619177
[900]   valid_set's binary_logloss: 0.0613089
[950]   valid_set's binary_logloss: 0.0607002
[1000]  valid_set's binary_logloss: 0.0601336
[1050]  valid_set's binary_logloss: 0.0596524
[1100]  valid_set's binary_logloss: 0.

In [20]:
test_identity = pd.read_csv('ieee-fraud-detection/test_identity.csv')
test_transaction = pd.read_csv('ieee-fraud-detection/test_transaction.csv')
test_data = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')  # same join applied to training files

y_predproba = predictor.predict_proba(test_data)
y_predproba.head(5)  # some example predicted fraud-probabilities